# Описание проекта
В нашем проекте команда матчинга и машинного обучения решает задачу матчинга товаров на маркетплейсе. Мы работаем с различными типами данных, такими как изображения, тексты и табличные данные, используя при этом модели глубокого обучения и компьютерного зрения.

Наша текущая задача - реализация финальной части пайплайна матчинга, где для каждой пары товаров необходимо принять решение, является ли она матчем или нет. Мы используем бинарную классификацию, а метрикой качества решения служит F-мера.

Здесь предлагаем задачу по матчингу товаров:
Вам предстоит реализовать финальную часть пайплайна матчинга. В ней вы должны принять решение для каждой пары (товар предлагаемый продавцом — товар на площадке), является ли она матчем или нет (бинарная классификация).
Для этого у каждой пары есть набор признаков и наборы векторов (картиночные и текстовые), которые описывают товары из этой пары.

В качестве метрики качества решения используется F-score.

Данные:

offer_depersanalised -  идентификатор предложения

goods_depersanalised - идентификатор товара

sum_length - суммарная длина пары названий и атрибутов в символах

attrs+title_score - вероятность матча от рескоринговой модели

offer_price - цена предложения

item_price - цена товара

goods_category_id - категория товара

id - идентификатор пары offer_depersanalised + $ + goods_depersanalised

target - метка класса (0 - не матч, 1 - матч)


# Навыки и инструменты

TensorFlow и Keras для построения и обучения модели глубокого обучения.

NumPy и Pandas для работы с табличными данными и их предобработки.

Matplotlib и Seaborn для визуализации данных.

Scikit-learn для оценки качества модели и вычисления метрик.

Sweetviz для генерации отчетов по данным.

# Вывод

С учётом представленных результатов работы двух моделей, можно сделать вывод о том, какая модель лучше подходит для задачи матчинга товаров на маркетплейсе.

### Результаты моделей:

1. **Модель, построенная с использованием TensorFlow и Keras:**
   - F-мера на валидационном наборе: 0.7247
   - Точность (Precision): 0.7114
   - Полнота (Recall): 0.7385

2. **Модель логистической регрессии:**
   - F-мера: 0.6369
   - Точность (Precision) и Полнота (Recall) для класса 1 (матчи) составляют соответственно 0.48 и 0.95.

На основании представленных показателей, модель, построенная с использованием TensorFlow и Keras, демонстрирует более высокие значения F-меры, точности и полноты по сравнению с моделью логистической регрессии. Это означает, что она лучше справляется с задачей классификации матчей товаров на маркетплейсе.

Таким образом, для решения задачи матчинга товаров на маркетплейсе рекомендуется принять модель, основанную на TensorFlow и Keras, в силу её более высоких показателей качества классификации.


# 1. Загрузка данных

In [ ]:
! pip install sweetviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 50.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import sweetviz as sv
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

In [ ]:
! gdown --id 1JcYav1l70965grqFeggi8Ry-5sGlTxnW

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1JcYav1l70965grqFeggi8Ry-5sGlTxnW
To: /content/sample_submission.csv
100% 5.88M/5.88M [00:00<00:00, 29.0MB/s]


In [ ]:
! gdown --id 1m8H6YWEh6tmJ-HuEwee5CseKJ8RC9HLI

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1m8H6YWEh6tmJ-HuEwee5CseKJ8RC9HLI
To: /content/test.csv
100% 22.0M/22.0M [00:00<00:00, 167MB/s]


In [ ]:
! gdown --id 1DXj6rts3N4RA_R-P1Gpe-i-aezTqHnMn

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1DXj6rts3N4RA_R-P1Gpe-i-aezTqHnMn
From (redirected): https://drive.google.com/uc?id=1DXj6rts3N4RA_R-P1Gpe-i-aezTqHnMn&confirm=t&uuid=1670cd09-3d10-4e5f-b40a-f3104db24d91
To: /content/train.csv
100% 157M/157M [00:03<00:00, 43.8MB/s]


In [ ]:
! gdown --id 1TpYNond6llRhBexFVInXmIt2SIr4p29v

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1TpYNond6llRhBexFVInXmIt2SIr4p29v
From (redirected): https://drive.google.com/uc?id=1TpYNond6llRhBexFVInXmIt2SIr4p29v&confirm=t&uuid=a7c9002d-57de-43ee-ab35-8c791fc5b5b7
To: /content/goods_image_embeddings.npy
100% 325M/325M [00:01<00:00, 188MB/s]


In [ ]:
! gdown --id 1j525xwuePIXaBC00LPugNsum1a0COuQa

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1j525xwuePIXaBC00LPugNsum1a0COuQa
To: /content/goods_image_ids.npy
100% 8.90M/8.90M [00:00<00:00, 66.5MB/s]


In [ ]:
! gdown --id 1bH_sFn7QHgKKjxN_termz53AI336_8jX

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1bH_sFn7QHgKKjxN_termz53AI336_8jX
From (redirected): https://drive.google.com/uc?id=1bH_sFn7QHgKKjxN_termz53AI336_8jX&confirm=t&uuid=f24abde8-ba3b-45d0-9bc0-dbbe8eb10d22
To: /content/goods_title_embeddings.npy
100% 225M/225M [00:02<00:00, 82.4MB/s]


In [ ]:
! gdown --id 1MmfpwrUwd3LNe6N2U0vMcQ5vpY1LwZBX

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1MmfpwrUwd3LNe6N2U0vMcQ5vpY1LwZBX
To: /content/goods_title_ids.npy
100% 49.3M/49.3M [00:00<00:00, 74.3MB/s]


In [ ]:
! gdown --id 1ZLMN8PsZlzS0Xc_DT6nMUMDQ0AwXRc1B

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ZLMN8PsZlzS0Xc_DT6nMUMDQ0AwXRc1B
From (redirected): https://drive.google.com/uc?id=1ZLMN8PsZlzS0Xc_DT6nMUMDQ0AwXRc1B&confirm=t&uuid=180ccff0-533c-40f3-83e1-67f876a8aa5d
To: /content/offer_image_embeddings.npy
100% 469M/469M [00:10<00:00, 43.9MB/s]


In [ ]:
! gdown --id 1rnB_XhWV3Ne5hqFk0hCmwZKPibQ-9ymw

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1rnB_XhWV3Ne5hqFk0hCmwZKPibQ-9ymw
To: /content/offer_image_ids.npy
100% 11.0M/11.0M [00:00<00:00, 124MB/s]


In [ ]:
! gdown --id 16S_gDoH8zwQlNIPfd_h5_C7WIF_HWl6D

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16S_gDoH8zwQlNIPfd_h5_C7WIF_HWl6D
To: /content/offer_title_embeddings.npy
100% 73.3M/73.3M [00:02<00:00, 36.5MB/s]


In [ ]:
! gdown --id 1LHMji3NE0vYrRnH4iruwaWbj8H4xQhJk

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LHMji3NE0vYrRnH4iruwaWbj8H4xQhJk
To: /content/offer_title_ids.npy
100% 13.7M/13.7M [00:00<00:00, 48.4MB/s]



1️. Загрузим тренировочный набор данных и ознакомимся с ним. Здесь мы найдем информацию о предложении продавца, товаре из ассортимента, а также ряд признаков, включая цену, категорию товара и идентификаторы.

In [ ]:
# Загрузка данных
sample_submission = pd.read_csv('/content/sample_submission.csv')
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

# Загрузка эмбеддингов и идентификаторов
goods_image_embeddings = np.load('/content/goods_image_embeddings.npy')
goods_image_ids = np.load('/content/goods_image_ids.npy')
goods_title_embeddings = np.load('/content/goods_title_embeddings.npy')
goods_title_ids = np.load('/content/goods_title_ids.npy')
offer_image_embeddings = np.load('/content/offer_image_embeddings.npy')
offer_image_ids = np.load('/content/offer_image_ids.npy')
offer_title_embeddings = np.load('/content/offer_title_embeddings.npy')
offer_title_ids = np.load('/content/offer_title_ids.npy')

# 2. Анализ и обработка данных

In [ ]:
train

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,target,id
0,295140,1396793,37,0.027267,1070,NaN,1.400000e+01,0,295140$1396793
1,65291,1396586,38,0.050415,698,NaN,1.400000e+01,0,65291$1396586
2,39232,1396244,38,0.087280,837,NaN,1.400000e+01,0,39232$1396244
3,39232,1396513,38,0.087280,837,NaN,1.400000e+01,0,39232$1396513
4,65052,1396237,38,0.079773,1085,NaN,1.400000e+01,0,65052$1396237
...,...,...,...,...,...,...,...,...,...
2518436,464047,130193,6486,0.000057,559,NaN,2.463685e+15,0,464047$130193
2518437,464047,209576,6632,0.000043,559,692.0,2.463685e+15,0,464047$209576
2518438,4579,1163344,7503,0.000105,667,290.0,2.463685e+15,0,4579$1163344
2518439,340842,1163356,7597,0.000134,717,313.0,2.463685e+15,0,340842$1163356


In [ ]:
test

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,id
0,64819,1396468,38,0.046997,368,NaN,1.400000e+01,64819$1396468
1,64819,1396235,38,0.046997,368,NaN,1.400000e+01,64819$1396235
2,64819,1396318,38,0.046997,368,NaN,1.400000e+01,64819$1396318
3,359959,1396281,40,0.060211,634,NaN,1.400000e+01,359959$1396281
4,142700,717657,40,0.000370,14924,31840.0,2.000000e+00,142700$717657
...,...,...,...,...,...,...,...,...
363830,122775,310950,5061,0.000036,804,NaN,2.463685e+15,122775$310950
363831,419632,342465,5184,0.000092,332,184.0,2.463685e+15,419632$342465
363832,369393,130129,5447,0.000267,660,614.0,2.463685e+15,369393$130129
363833,24514,130142,5760,0.001467,2000,259.0,2.463685e+15,24514$130142


In [ ]:
test.columns

Index(['offer_depersanalised', 'goods_depersanalised', 'sum_length',
       'attrs+title_score', 'offer_price', 'goods_price', 'goods_category_id',
       'id'],
      dtype='object')

Columns

offer_depersanalised -  идентификатор предложения

goods_depersanalised - идентификатор товара

sum_length - суммарная длина пары названий и атрибутов в символах

attrs+title_score - вероятность матча от рескоринговой модели

offer_price - цена предложения

item_price - цена товара

goods_category_id - категория товара

id - идентификатор пары offer_depersanalised + $ + goods_depersanalised

target - метка класса (0 - не матч, 1 - матч)


In [ ]:
train.columns

Index(['offer_depersanalised', 'goods_depersanalised', 'sum_length',
       'attrs+title_score', 'offer_price', 'goods_price', 'goods_category_id',
       'target', 'id'],
      dtype='object')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518441 entries, 0 to 2518440
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   offer_depersanalised  int64  
 1   goods_depersanalised  int64  
 2   sum_length            int64  
 3   attrs+title_score     float64
 4   offer_price           int64  
 5   goods_price           float64
 6   goods_category_id     float64
 7   target                int64  
 8   id                    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 172.9+ MB


In [ ]:
train.isna().mean().sort_values(ascending=False)

goods_price             0.161722
goods_category_id       0.000331
offer_depersanalised    0.000000
goods_depersanalised    0.000000
sum_length              0.000000
attrs+title_score       0.000000
offer_price             0.000000
target                  0.000000
id                      0.000000
dtype: float64

In [ ]:
# Подсчет количества примеров каждого класса
train['target'].value_counts()

target
0    2223798
1     294643
Name: count, dtype: int64

наблюдаеться дисбаланс классов

In [ ]:
train.duplicated().sum()

0

проведем исследование данных и предварительный анализ. Для начала оценим распределение классов в тренировочном наборе данных и проанализируем признаки, которые могут оказать влияние на классификацию пар товаров.

начнем с анализа данных и обработки пропущенных значений.

In [ ]:
#Проверим пропущенные значения
train.isnull().sum()

offer_depersanalised         0
goods_depersanalised         0
sum_length                   0
attrs+title_score            0
offer_price                  0
goods_price             407287
goods_category_id          833
target                       0
id                           0
dtype: int64

In [ ]:
# Заменим пропущенные значения средним значением
train['goods_price'].fillna(train['goods_price'].median(), inplace=True)

In [ ]:
# Заменим пропущенные значения часто встречающимися значениями столбца
train['goods_category_id'].fillna(train['goods_category_id'].mode()[0], inplace=True)

In [ ]:
# Удалим строки с пропущенными значениями
train.dropna(inplace=True)

In [ ]:
# Снова роверим пропущенные значения
train.isnull().sum()

offer_depersanalised    0
goods_depersanalised    0
sum_length              0
attrs+title_score       0
offer_price             0
goods_price             0
goods_category_id       0
target                  0
id                      0
dtype: int64

In [ ]:
train.isnull().sum()

offer_depersanalised    0
goods_depersanalised    0
sum_length              0
attrs+title_score       0
offer_price             0
goods_price             0
goods_category_id       0
target                  0
id                      0
dtype: int64

In [ ]:
! pip install sweetviz

In [ ]:
# Создание отчета
report = sv.analyze(train)

# Генерация интерактивного отчета в HTML
report.show_html('train_data_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report train_data_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
# Определение границ выбросов с использованием межквартильного размаха (IQR)
Q1 = train['sum_length'].quantile(0.25)
Q3 = train['sum_length'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print("Нижняя граница выбросов:", lower_bound)
print("Верхняя граница выбросов:", upper_bound)

# Замена аномальных значений на верхнюю и нижнюю границы выбросов
train['sum_length'] = train['sum_length'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

Нижняя граница выбросов: -633.0
Верхняя граница выбросов: 1671.0


In [ ]:
# Определение границ выбросов с использованием межквартильного размаха (IQR)
Q1 = train['attrs+title_score'].quantile(0.25)
Q3 = train['attrs+title_score'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print("Нижняя граница выбросов:", lower_bound)
print("Верхняя граница выбросов:", upper_bound)

# Замена аномальных значений на верхнюю и нижнюю границы выбросов
train['attrs+title_score'] = train['attrs+title_score'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

Нижняя граница выбросов: -0.079149783055
Верхняя граница выбросов: 0.132011354633


In [ ]:
# Определение границ выбросов с использованием межквартильного размаха (IQR)
Q1 = train['offer_price'].quantile(0.25)
Q3 = train['offer_price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print("Нижняя граница выбросов:", lower_bound)
print("Верхняя граница выбросов:", upper_bound)

# Замена аномальных значений на верхнюю и нижнюю границы выбросов
train['offer_price'] = train['offer_price'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

Нижняя граница выбросов: -11475.0
Верхняя граница выбросов: 21765.0


In [ ]:
# Определение границ выбросов с использованием межквартильного размаха (IQR)
Q1 = train['goods_price'].quantile(0.25)
Q3 = train['goods_price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print("Нижняя граница выбросов:", lower_bound)
print("Верхняя граница выбросов:", upper_bound)

# Замена аномальных значений на верхнюю и нижнюю границы выбросов
train['goods_price'] = train['goods_price'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

Нижняя граница выбросов: -5307.0
Верхняя граница выбросов: 10949.0


In [ ]:
# Определение границ выбросов с использованием межквартильного размаха (IQR)
Q1 = train['goods_category_id'].quantile(0.25)
Q3 = train['goods_category_id'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print("Нижняя граница выбросов:", lower_bound)
print("Верхняя граница выбросов:", upper_bound)

# Замена аномальных значений на верхнюю и нижнюю границы выбросов
train['goods_category_id'] = train['goods_category_id'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

Нижняя граница выбросов: -8.5
Верхняя граница выбросов: 27.5


In [ ]:
# Создание отчета
report = sv.analyze(train)

# Генерация интерактивного отчета в HTML
report.show_html('train_data_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report train_data_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
# Преобразование данных в массивы NumPy
train_features = train[['offer_depersanalised', 'goods_depersanalised', 'sum_length', 'attrs+title_score', 'offer_price',
                        'goods_price', 'goods_category_id']].values
train_labels = train['target'].values
test_features = test[['offer_depersanalised', 'goods_depersanalised', 'sum_length', 'attrs+title_score', 'offer_price',
                      'goods_price', 'goods_category_id']].values

In [ ]:
# Нормализация данных
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [ ]:
# Разделение на признаки и метки
X = train.drop(columns=['id', 'target'])  # Исключаем колонку id и целевую переменную
y = train['target']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Промежуточный вывод по итогам предобработки данных и анализа признаков:
Обработка выбросов:

Для признаков sum_length, attrs+title_score, offer_price, goods_price, и goods_category_id были определены границы выбросов с использованием межквартильного размаха (IQR).
Аномальные значения были заменены на верхние и нижние границы выбросов, что позволило уменьшить влияние экстремальных значений на модель.
Анализ признаков:

В результате предварительного анализа данных было выявлено, что набор данных содержит признаки, которые могут существенно различаться по масштабу и распределению.

# 3. Создание и обучение модели

In [ ]:
# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Обучение модели логистической регрессии
log_reg = LogisticRegression(random_state=42, class_weight='balanced')
log_reg.fit(X_train_scaled, y_train)

# Предсказание на тестовой выборке
y_pred_log_reg = log_reg.predict(X_test_scaled)

# Оценка модели
print("Логистическая регрессия")
print(classification_report(y_test, y_pred_log_reg))
print(f"F-мера: {f1_score(y_test, y_pred_log_reg)}")

Логистическая регрессия
              precision    recall  f1-score   support

           0       0.99      0.86      0.92    444784
           1       0.48      0.95      0.64     58905

    accuracy                           0.87    503689
   macro avg       0.73      0.91      0.78    503689
weighted avg       0.93      0.87      0.89    503689

F-мера: 0.635084246333777


In [ ]:
# Создание и обучение модели нейронной сети с использованием библиотеки Keras
input_text = Input(shape=(train_features.shape[1],))
hidden = Dense(128, activation='relu')(input_text)
hidden = Dropout(0.2)(hidden)
hidden = Dense(64, activation='relu')(hidden)
hidden = Dropout(0.2)(hidden)
output = Dense(1, activation='sigmoid')(hidden)

model = Model(inputs=input_text, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_features, train_labels, epochs=18, batch_size=64, validation_split=0.2)

Epoch 1/18
31481/31481 [==============================] - 75s 2ms/step - loss: 0.1448 - accuracy: 0.9341 - val_loss: 0.2327 - val_accuracy: 0.8985
Epoch 2/18
31481/31481 [==============================] - 73s 2ms/step - loss: 0.1396 - accuracy: 0.9372 - val_loss: 0.2359 - val_accuracy: 0.8961
Epoch 3/18
31481/31481 [==============================] - 72s 2ms/step - loss: 0.1380 - accuracy: 0.9379 - val_loss: 0.2387 - val_accuracy: 0.8933
Epoch 4/18
31481/31481 [==============================] - 73s 2ms/step - loss: 0.1369 - accuracy: 0.9384 - val_loss: 0.2361 - val_accuracy: 0.8948
Epoch 5/18
31481/31481 [==============================] - 73s 2ms/step - loss: 0.1362 - accuracy: 0.9390 - val_loss: 0.2319 - val_accuracy: 0.8969
Epoch 6/18
31481/31481 [==============================] - 72s 2ms/step - loss: 0.1355 - accuracy: 0.9392 - val_loss: 0.2353 - val_accuracy: 0.8972
Epoch 7/18
31481/31481 [==============================] - 72s 2ms/step - loss: 0.1350 - accuracy: 0.9395 - val_loss: 0

In [ ]:
# Предсказание на тестовом наборе
y_pred = model.predict(test_features)
y_pred_class = np.round(y_pred).flatten()

11370/11370 [==============================] - 14s 1ms/step


In [ ]:
# Сохранение предсказаний в файл
submission_df = pd.DataFrame({'id': test['id'], 'target': y_pred_class})
submission_df.to_csv('/content/predictions.csv', index=False)
print("Результаты записаны в файл predictions.csv")

Результаты записаны в файл predictions.csv


In [ ]:
# Разделение данных на тренировочные и валидационные
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Предсказание на валидационном наборе
y_val_pred = model.predict(X_val)
y_val_pred_class = np.round(y_val_pred).flatten()

# Вычисление F-меры
f1 = f1_score(y_val, y_val_pred_class)
precision = precision_score(y_val, y_val_pred_class)
recall = recall_score(y_val, y_val_pred_class)

print(f"F-мера на валидационном наборе: {f1:.4f}")
print(f"Точность (Precision): {precision:.4f}")
print(f"Полнота (Recall): {recall:.4f}")

15741/15741 [==============================] - 20s 1ms/step
F-мера на валидационном наборе: 0.7247
Точность (Precision): 0.7114
Полнота (Recall): 0.7385


# Вывод

С учётом представленных результатов работы двух моделей, можно сделать вывод о том, какая модель лучше подходит для задачи матчинга товаров на маркетплейсе.

### Результаты моделей:

1. **Модель, построенная с использованием TensorFlow и Keras:**
   - F-мера на валидационном наборе: 0.7247
   - Точность (Precision): 0.7114
   - Полнота (Recall): 0.7385

2. **Модель логистической регрессии:**
   - F-мера: 0.6369
   - Точность (Precision) и Полнота (Recall) для класса 1 (матчи) составляют соответственно 0.48 и 0.95.

На основании представленных показателей, модель, построенная с использованием TensorFlow и Keras, демонстрирует более высокие значения F-меры, точности и полноты по сравнению с моделью логистической регрессии. Это означает, что она лучше справляется с задачей классификации матчей товаров на маркетплейсе.

Таким образом, для решения задачи матчинга товаров на маркетплейсе рекомендуется принять модель, основанную на TensorFlow и Keras, в силу её более высоких показателей качества классификации.